In [1]:
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

# Hàm để crawl dữ liệu từ trang web và trích xuất thông tin cần thiết
def crawl_data(url):
    # Gửi yêu cầu GET đến trang web
    response = requests.get(url)
    
    # Kiểm tra xem yêu cầu có thành công hay không
    if response.status_code == 200:
        # Sử dụng BeautifulSoup để phân tích nội dung HTML của trang web
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Tìm các phần tử chứa thông tin cần thiết
        # Phần tử đầu tiên chứa bảng thông tin doanh thu
        revenue_table = soup.find('table', {'id': 'tableContent'})
        revenue_data = extract_table_data(revenue_table)
        
        # Phần tử thứ hai chứa bảng thông tin các khoản giảm trừ doanh thu
        deduction_table = soup.find_all('table', {'class': 'BaoCaoTaiChinh_Chart'})[1]
        deduction_data = extract_table_data(deduction_table)
        
        # Trả về dữ liệu đã trích xuất
        return revenue_data, deduction_data
    else:
        print("Yêu cầu không thành công")
        return None, None

# Hàm để trích xuất dữ liệu từ bảng HTML
def extract_table_data(table):
    data = []
    if table:
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all(['td', 'th'])
            row_data = []
            for col in cols:
                col_text = col.text.strip()
                row_data.append(col_text)
            data.append(row_data)
    return data

# Hàm để tạo URL cho một năm cụ thể
def generate_url(companyName, year):
    return f'https://s.cafef.vn/bao-cao-tai-chinh/{companyName}/cashflow/{year}/4/0/0/luu-chuyen-tien-te-truc-tiep-.chn'

def export_data_to_excel(companyName):
    # Lặp qua các năm từ 2019 đến 2023
    for year in range(2019, 2024):
        # Tạo URL cho năm hiện tại
        url = generate_url(companyName, year)
        
        # Tạo một Workbook mới cho mỗi năm
        wb = Workbook()
        
        # Gọi hàm crawl_data để trích xuất dữ liệu từ trang web
        revenue_data, deduction_data = crawl_data(url)
        
        # Tạo một sheet mới và ghi dữ liệu vào đó
        revenue_sheet = wb.active
        revenue_sheet.title = f"Lưu chuyển tiền tệ trực tiếp {year}"
        revenue_sheet.append([]) # Thêm một dòng trống ở đầu
        for row in revenue_data:
            revenue_sheet.append(row if row else [""] * len(revenue_data[0]))  # Ensure empty rows are written

        # Lưu Workbook vào một file Excel với tên tương ứng với năm
        wb.save(f"[{companyName}] LCTT_truc_tiep_{year}.xlsx")


In [2]:
import pandas as pd
import os

def merge_file_quarter(companyName, quarter):
    # Đường dẫn đến thư mục chứa các file Excel
    folder_path = 'C:/Users/Lenovo/Desktop/project'

    # Đọc dữ liệu từ file financial_data_2019.xlsx
    file_2019 = os.path.join(folder_path, f'[{companyName}] LCTT_truc_tiep_2019.xlsx')
    df_2019 = pd.read_excel(file_2019)

    # Lấy cột đầu tiên từ file financial_data_2019.xlsx
    df_2019 = df_2019.iloc[:, :1]

    # Khởi tạo DataFrame mới để chứa dữ liệu
    df_combined = df_2019.copy()

    # Đọc dữ liệu từ các file Excel và lấy cột thứ 2 tat ca cac file
    for year in range(2019, 2024):
        file_path = os.path.join(folder_path, f'[{companyName}] LCTT_truc_tiep_{year}.xlsx')
        df_year = pd.read_excel(file_path)
        df_combined[f'Year - {year}'] = df_year.iloc[:, quarter]

    # Thêm header cho dòng đầu tiên
    header_row = ['']
    for year in range(2019, 2024):
     header_row.append(f'Quarter {quarter} - {year}')
    df_combined.columns = header_row

    # Lưu DataFrame kết hợp vào file excel mới
    output_file = os.path.join(folder_path, f'[{companyName}] Quy{quarter}_LCTT_truc_tiep.xlsx')
    df_combined.to_excel(output_file, index=False)

In [3]:
export_data_to_excel('aav')
for i in range(1,5):
    merge_file_quarter('aav',i)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
